In [19]:
import pandas as pd
from portfolios.portfolio_new import Portfolio
from portfolios.FF5FM_Mom import FF5FM_Mom

In [15]:
results_df = pd.read_csv('/home/ge65cuw/thesis/saved/final_results/results_df.csv', index_col=0).dropna()
worst_net_id = results_df.loc[results_df.trial_id == results_df.trial_id.min()]['trial_id'].values[0]

In [17]:
base_path = '/home/ge65cuw/thesis/saved/final_results/'
best_net_path = base_path + 'ensembleNN3.csv'
worst_net_path = base_path + 'predicted_returns/' + str(worst_net_id) + '_predicted_returns.csv'
benchmark_nn4_path = base_path + 'ensembleGu.csv'
linear_path = base_path + 'linear/predicted_ret.csv'
# market_returns = ''

In [18]:
df = pd.read_csv(best_net_path, index_col=0)
pbest = Portfolio(df)

Loading the dataset...
Features stats:
Mean: -0.01	Std: 0.29	Median: -0.02
IR & alpha calculation...
Params
const     1.390458
Mkt-RF   -0.045262
SMB      -0.157068
HML       0.346980
RMW       0.138406
CMA       0.260275
Mom       0.018406
ST_Rev   -0.006553
dtype: float64
tValues
const     6.178562
Mkt-RF   -0.767920
SMB      -2.072170
HML       3.669293
RMW       1.395709
CMA       1.888540
Mom       0.378823
ST_Rev   -0.105896
dtype: float64
Linear Regression summary:
                            OLS Regression Results                            
Dep. Variable:                    l-s   R-squared:                       0.228
Model:                            OLS   Adj. R-squared:                  0.209
Method:                 Least Squares   F-statistic:                     12.13
Date:                Sat, 19 Nov 2022   Prob (F-statistic):           1.41e-13
Time:                        07:58:51   Log-Likelihood:                -790.85
No. Observations:                 296   AIC:     

In [ ]:
portfolio = pbest
FFMom = FF5FM_Mom().returns
columns_to_keep = [portfolio.returns.columns[0]]
columns_to_keep.extend(portfolio.returns.columns[-2:])
df = portfolio.returns[columns_to_keep]
df = df.merge(FFMom, on=['yyyymm'], how='left')


# On long-short returns        
# X = df.iloc[:, 3:]

# Column 1 is long returns on max quantile, 
# Column 2 is long-short returns
# if method == 'long-short':
#     y = df.iloc[:,2]
# elif method == 'long':
#     y = df.iloc[:,1]

X = sm.add_constant(X)
lm = sm.OLS(y, X).fit()

print('Params')
print(lm.params)
print('tValues')
print(lm.tvalues)

alpha = lm.params[0]
t_value_alpha = lm.tvalues[0]
regression_rsquared = lm.rsquared
print('Linear Regression summary:')
print(lm.summary())


year_min = portfolio.returns['yyyymm'].min()//100
month_min = portfolio.returns['yyyymm'].min()%100

year_max = portfolio.returns['yyyymm'].max()//100
month_max = portfolio.returns['yyyymm'].max()%100

n_months =  ((year_max - year_min) + (month_max-month_min+1)/12)*12 
information_ratio_regression = t_value_alpha / np.sqrt(n_months)
print(f'Informatio Ratio based on FF5FM+Mom regression: {information_ratio_regression:.2f}')